# Creating and Loading tables from csv files

In [ ]:
import csv
import pprint
data_list = [
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/BusStops.csv'))),
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/LibraryBookstore.csv'))),
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/ParksPlaygrounds.csv'))),
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/TrainStation.csv')))
]

In [ ]:
def pgexec( conn, sqlcmd, args, msg, silent=False ):
   """ utility function to execute some SQL statement
       can take optional arguments to fill in (dictionary)
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
               cur.execute(sqlcmd)
            else:
               cur.execute(sqlcmd, args)
            if silent == False: 
                print("success: " + msg)
            retval = True
         except Exception as e:
            if silent == False: 
                print("db error: ")
                print(e)
   return retval

In [ ]:
def pgquery( conn, sqlcmd, args, silent=False ):
   """ utility function to execute some SQL query statement
       can take optional arguments to fill in (dictionary)
       will print out on screen the result set of the query
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
                cur.execute(sqlcmd)
            else:
                cur.execute(sqlcmd, args)
            if silent == False:
                for record in cur:
                    print(record)
            retval = True
         except Exception as e:
            if silent == False:
                print("db read error: ")
                print(e)
   return retval

In [ ]:
import psycopg2

def pgconnect():
    # please replace <your_unikey> and <your_SID> with your own details
    YOUR_UNIKEY = 'hohe6356'#'<your_unikey>'
    YOUR_PW     = '460012791'#'<your_SID>'
    try: 
        conn = psycopg2.connect(host='soit-db-pro-1.ucc.usyd.edu.au',
                               database='y18s1d2001_'+YOUR_UNIKEY,
                               user='y18s1d2001_'+YOUR_UNIKEY, 
                               password=YOUR_PW)
        
#         conn = psycopg2.connect(host='localhost', database='waterinfo', user='matloob', password='matloob')
        
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
    return conn
pgconnect()


In [ ]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, 'DROP TABLE IF EXISTS walkability.BusStops',None, "Reset Table BusStops")
pgexec (conn, 'DROP TABLE IF EXISTS walkability.LibraryBookstore',None, "Reset Table LibraryBookstore")
pgexec (conn, 'DROP TABLE IF EXISTS walkability.ParksPlaygrounds',None, "Reset Table ParksPlaygrounds")
pgexec (conn, 'DROP TABLE IF EXISTS walkability.TrainStation',None, "Reset Table TrainStation")

# 2nd: ensure that the schema is in place
BusStops_schema = """CREATE TABLE IF NOT EXISTS walkability.BusStops (
                        stop_id BIGINT PRIMARY KEY,
                        stop_code TEXT,
                        gps POINT
                   )"""
LibraryBookstore_schema = """CREATE TABLE IF NOT EXISTS walkability.LibraryBookstore (
                        name TEXT PRIMARY KEY,
                        gps POINT
                   )"""
ParksPlaygrounds_schema = """CREATE TABLE IF NOT EXISTS walkability.ParksPlaygrounds (
                        name TEXT PRIMARY KEY,
                        gps POINT
                   )"""
TrainStation_schema = """CREATE TABLE IF NOT EXISTS walkability.TrainStation (
                        name TEXT PRIMARY KEY,
                        gps POINT
                   )"""

pgexec (conn, BusStops_schema, None, 'Create Table BusStops')
pgexec (conn, LibraryBookstore_schema, None, 'Create Table LibraryBookstore')
pgexec (conn, ParksPlaygrounds_schema, None, 'Create Table ParksPlaygrounds')
pgexec (conn, TrainStation_schema, None, 'Create Table TrainStation')

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = [
    """INSERT INTO walkability.BusStops(stop_id,stop_code,gps) VALUES (%(stop_id)s,%(stop_code)s,POINT(%(stop_lat)s,%(stop_lon)s))""",
    """INSERT INTO walkability.LibraryBookstore(name,gps) VALUES (%(name)s,POINT(%(GPS_lat)s,%(GPS_lng)s))""",
    """INSERT INTO walkability.ParksPlaygrounds(name,gps) VALUES (%(name)s,POINT(%(GPS_lat)s,%(GPS_lng)s))""",
    """INSERT INTO walkability.TrainStation(name,gps) VALUES (%(name)s,POINT(%(GPS_lat)s,%(GPS_lng)s))"""
]
for insert,data in zip(insert_stmt,data_list):
    for row in data:
         pgexec (conn, insert, row, "row inserted")

# Creating and Loading a table from a txt file

In [ ]:
# Step 1. Create a table caleed busStops
conn = pgconnect()
# pgexec (conn, 'DROP TABLE IF EXISTS walkability.BusStops',None, "Reset Table BusStops")
Stops_schema = """CREATE TABLE IF NOT EXISTS walkability.busStops (
                        stop_id TEXT PRIMARY KEY,
                        stop_code TEXT,
                        stop_name TEXT,
                        stop_lat FLOAT,
                        stop_lon FLOAT,
                        location_type TEXT,
                        parent_station TEXT,
                        wheelchair_boarding TEXT,
                        platform_code TEXT
                   )"""
# pgexec (conn, Stops_schema, None, 'Create Table Stops')

# Step 2. Load data into busStops table from dataset 
# This is done by typing the command below in the terminal 
query_stmt = """\copy walkability.stops FROM '/home/hohe6356/Desktop/stops.txt' WITH CSV HEADER"""

# Step 3. Drop the useless columns
sqlcmd = """ALTER TABLE walkability.BusStops 
            DROP COLUMN stop_code, DROP COLUMN location_type, DROP COLUMN parent_station,DROP COLUMN wheelchair_boarding,DROP COLUMN platform_code;"""
pgexec (conn, sqlcmd, row, "Drop useless columns")

